In [1]:
!pip install transformers
!pip install datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

datasets = load_dataset("cifar10")

labels = datasets["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
print(datasets["train"].features["label"].names)
print(label2id)
print(id2label)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [5]:
import torch
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [6]:
import torch.optim as optim
import torch.nn.functional as F
import time

In [9]:
def homemade_batch(num_img, batch_size=10, start_img=0, data_type = "test"):
    homemade = []
    num_batches = num_img // batch_size
    extra = num_img % batch_size

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    t_0 = time.perf_counter()

    for i in range(num_batches):
        t1 = time.perf_counter()

        subset = datasets["test"].select(range((i * batch_size + start_img), (i + 1) * batch_size + start_img))
        input = processor(text=labels, images=subset["img"], return_tensors="pt", padding=False).to(device)
        output = model(**input)

        logits = output.logits_per_image
        probs = logits.softmax(dim=1)

        max_prob, max_index = probs.max(dim=1)
        homemade.append([max_prob.cpu().detach(), max_index.cpu().detach()])
        torch.cuda.empty_cache()

        t2 = time.perf_counter()
        print(f"Finished batch {i + 1} of {num_batches} in {t2 - t1} seconds")

    t_3 = time.perf_counter()
    print(f"Finished entire dataset in {t_3 - t_0} seconds")

    return homemade

In [15]:
def prediction_reformat(subset):
    predicted = []
    count = 1
    for i in range(len(subset)):
        for k in range(len(subset[0][0])):
            prob = subset[i][0][k].item()
            id = subset[i][1][k].item()
            label = id2label[id]
            predicted.append([i, label, prob, id])

            count += 1

    return predicted

In [18]:
def accuracy(result, data_type = "train"):
    correct = 0
    total = 0

    all_labels = {}
    for label in datasets[data_type].features["label"].names:
        all_labels[label] = 0

    incorrect = {}
    for label in datasets[data_type].features["label"].names:
        incorrect[label] = 0


    for i in range(len(result)):
        all_labels[result[i][1]] += 1
        if datasets[data_type][i]["label"] == result[i][3]:
            correct += 1
            total += 1
            if total % 50 == 0:
              print(f"Model accurately predicted {result[i][1]} with {result[i][2] * 100}% confidence.")
        else:
            total += 1
            print(f"Model inaccurately predicted {result[i][1]} with {result[i][2] * 100}% confidence.")
            incorrect[result[i][1]] += 1


    print(f"Acurracy: {(correct/total) * 100}%")

    worst_accuracy = []
    for label in all_labels:
        correct =  all_labels[label] - incorrect[label]
        total = all_labels[label]
        print(f"For {label}: Predicted {correct} out of {total} correct. {(correct) / total * 100}% Accuracy")
        worst_accuracy.append([label, correct/total])

    worst_group = min(worst_accuracy, key=lambda x: x[1])
    print(f"The worst performing group is '{worst_group[0]}' with an accuracy of {worst_group[1] * 100}%")

In [19]:
def data_analysis(predictions, data_type = "test"):
  cleaned = prediction_reformat(predictions)
  final_results = accuracy(cleaned, data_type)

In [13]:
pretrain = homemade_batch(5000)

Finished batch 1 of 500 in 2.6647961510000187 seconds
Finished batch 2 of 500 in 0.6593946360000018 seconds
Finished batch 3 of 500 in 0.6820709489999217 seconds
Finished batch 4 of 500 in 0.6896979519999604 seconds
Finished batch 5 of 500 in 0.6466695529999242 seconds
Finished batch 6 of 500 in 0.6485446650000313 seconds
Finished batch 7 of 500 in 0.65060516799997 seconds
Finished batch 8 of 500 in 0.6509827180000229 seconds
Finished batch 9 of 500 in 0.6781850090000034 seconds
Finished batch 10 of 500 in 0.6843413959999225 seconds
Finished batch 11 of 500 in 0.6822304489999169 seconds
Finished batch 12 of 500 in 0.6896571050000375 seconds
Finished batch 13 of 500 in 0.6829183060000332 seconds
Finished batch 14 of 500 in 0.6776793039999802 seconds
Finished batch 15 of 500 in 0.6445811170000297 seconds
Finished batch 16 of 500 in 0.654968978999932 seconds
Finished batch 17 of 500 in 0.6486007580000432 seconds
Finished batch 18 of 500 in 0.6593627289998949 seconds
Finished batch 19 of 5

In [20]:
data_analysis(pretrain)

Model inaccurately predicted horse with 18.26408952474594% confidence.
Model inaccurately predicted cat with 29.691317677497864% confidence.
Model inaccurately predicted truck with 74.78898763656616% confidence.
Model inaccurately predicted bird with 69.329172372818% confidence.
Model inaccurately predicted dog with 35.70859730243683% confidence.
Model inaccurately predicted dog with 41.98958873748779% confidence.
Model inaccurately predicted horse with 95.62273621559143% confidence.
Model accurately predicted frog with 99.14552569389343% confidence.
Model inaccurately predicted bird with 51.83928608894348% confidence.
Model inaccurately predicted deer with 37.0936781167984% confidence.
Model inaccurately predicted dog with 96.9225525856018% confidence.
Model inaccurately predicted dog with 60.95832586288452% confidence.
Model inaccurately predicted truck with 96.45574688911438% confidence.
Model accurately predicted horse with 98.48936200141907% confidence.
Model inaccurately predicte

In [7]:
def train(num_img, batch_size=10, num_epoch=2):
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    total_loss = 0
    model.to(device)
    model.train()

    for epoch in range(num_epoch):
        t0 = time.perf_counter()
        model.train()
        epoch_loss = 0


        for i in range(num_img // batch_size):
            train_set = datasets["train"].select(range(i * batch_size, (i + 1) * batch_size))
            t1 = time.perf_counter()
            input = processor(text=labels, images=train_set["img"], return_tensors="pt", padding=False).to(device)
            output = model(**input)

            logits_per_image = output.logits_per_image
            logits_per_text = output.logits_per_text
            logits_per_text = logits_per_text.squeeze()

            predicted_classes = torch.argmax(logits_per_image, dim=-1)
            predicted_classes_text = torch.argmax(logits_per_text, dim=0)

            targets = torch.tensor(train_set["label"]).to(device)

            loss_img = F.cross_entropy(logits_per_image, targets)
            loss_text = F.cross_entropy(logits_per_text, targets)
            loss = (loss_img + loss_text) / 2
            t2 = time.perf_counter()
            print(f"Finished batch {i + 1}/{num_img // batch_size} in {t2 - t1} seconds")

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        t3 = time.perf_counter()
        total_loss += epoch_loss
        avg_loss = epoch_loss / (num_img // batch_size)
        print(f"Epoch {epoch+1}/{num_epoch} completed in {t3 - t1} seconds, Loss: {avg_loss:.4f}")

In [8]:
def train_shuffle(num_img, batch_size=10, num_epoch=2):
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    total_loss = 0
    model.to(device)
    model.train()

    for epoch in range(num_epoch):
        t0 = time.perf_counter()
        model.train()
        epoch_loss = 0


        for i in range(num_img // batch_size):
            shuffled = datasets["train"].shuffle(seed=42)
            train_set = shuffled.select(range(i * batch_size, (i + 1) * batch_size))
            t1 = time.perf_counter()
            input = processor(text=labels, images=train_set["img"], return_tensors="pt", padding=False).to(device)
            output = model(**input)

            logits_per_image = output.logits_per_image
            logits_per_text = output.logits_per_text
            logits_per_text = logits_per_text.squeeze()

            predicted_classes = torch.argmax(logits_per_image, dim=-1)
            predicted_classes_text = torch.argmax(logits_per_text, dim=0)

            targets = torch.tensor(train_set["label"]).to(device)

            loss_img = F.cross_entropy(logits_per_image, targets)
            loss_text = F.cross_entropy(logits_per_text, targets)
            loss = (loss_img + loss_text) / 2
            t2 = time.perf_counter()
            print(f"Finished batch {i + 1} of {num_img // batch_size} in {t2 - t1} seconds")

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        t3 = time.perf_counter()
        total_loss += epoch_loss
        avg_loss = epoch_loss / (num_img // batch_size)
        print(f"Finished Epoch {epoch+1}/{num_epoch} in {t3 - t1} seconds, Loss: {avg_loss:.4f}")

In [26]:
train(5000)

Finished batch 1/500 in 0.6134511380000731 seconds
Finished batch 2/500 in 0.5245914550000634 seconds
Finished batch 3/500 in 0.5139969519998431 seconds
Finished batch 4/500 in 0.534709052999915 seconds
Finished batch 5/500 in 0.5341310659998726 seconds
Finished batch 6/500 in 0.5372866620000423 seconds
Finished batch 7/500 in 0.5434755040000709 seconds
Finished batch 8/500 in 0.5372407570000632 seconds
Finished batch 9/500 in 0.5347836820001248 seconds
Finished batch 10/500 in 0.5454707760000019 seconds
Finished batch 11/500 in 0.5525757959999282 seconds
Finished batch 12/500 in 0.5568598980000843 seconds
Finished batch 13/500 in 0.5599430990000656 seconds
Finished batch 14/500 in 0.5604578540001057 seconds
Finished batch 15/500 in 0.5672486410001056 seconds
Finished batch 16/500 in 0.5599743660000058 seconds
Finished batch 17/500 in 0.5595281339999474 seconds
Finished batch 18/500 in 0.5699490810000043 seconds
Finished batch 19/500 in 0.5347956850000628 seconds
Finished batch 20/500 

In [27]:
model.eval()

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [28]:
posttrain = homemade_batch(5000)


Finished batch 1 of 500 in 0.6773949720000019 seconds
Finished batch 2 of 500 in 0.6544980270000451 seconds
Finished batch 3 of 500 in 0.6472329600001103 seconds
Finished batch 4 of 500 in 0.6629204099999697 seconds
Finished batch 5 of 500 in 0.6431224339999062 seconds
Finished batch 6 of 500 in 0.6623176540001623 seconds
Finished batch 7 of 500 in 0.6483045420000053 seconds
Finished batch 8 of 500 in 0.6628992529999778 seconds
Finished batch 9 of 500 in 0.647271499999988 seconds
Finished batch 10 of 500 in 0.6643201729998509 seconds
Finished batch 11 of 500 in 0.6488309700002901 seconds
Finished batch 12 of 500 in 0.6727083460000358 seconds
Finished batch 13 of 500 in 0.6479908010001054 seconds
Finished batch 14 of 500 in 0.6661580740001227 seconds
Finished batch 15 of 500 in 0.6806059939999614 seconds
Finished batch 16 of 500 in 0.6862783579999814 seconds
Finished batch 17 of 500 in 0.6678913499999908 seconds
Finished batch 18 of 500 in 0.6831735139999182 seconds
Finished batch 19 of

In [29]:
data_analysis(posttrain)

Model inaccurately predicted automobile with 50.38509964942932% confidence.
Model inaccurately predicted automobile with 53.13445329666138% confidence.
Model inaccurately predicted truck with 24.042566120624542% confidence.
Model accurately predicted frog with 55.79007863998413% confidence.
Model inaccurately predicted dog with 45.80007493495941% confidence.
Model inaccurately predicted cat with 32.360830903053284% confidence.
Model inaccurately predicted dog with 66.42454862594604% confidence.
Model inaccurately predicted dog with 32.553380727767944% confidence.
Model accurately predicted horse with 61.051297187805176% confidence.
Model inaccurately predicted deer with 21.36949747800827% confidence.
Model inaccurately predicted dog with 62.57783770561218% confidence.
Model accurately predicted bird with 41.380733251571655% confidence.
Model inaccurately predicted airplane with 55.87933659553528% confidence.
Model inaccurately predicted airplane with 58.18147659301758% confidence.
Mode